In [ ]:
import os
from langchain.agents import AgentType, initialize_agent, create_pandas_dataframe_agent
from langchain.agents.agent_toolkits import create_vectorstore_agent, VectorStoreToolkit, VectorStoreInfo, create_python_agent
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.tools import HumanInputRun
from langchain.tools import Tool
from langchain.tools.python.tool import PythonREPLTool
from langchain.utilities import SerpAPIWrapper
from langchain.vectorstores import Chroma

import pandas as pd

In [ ]:
from getpass import getpass
import os

OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY



In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)


In [ ]:
# simple test
from langchain.llms import OpenAI
from langchain import PromptTemplate, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
df = pd.read_csv("samples/data.csv")
loader = DataFrameLoader(df, page_content_column="description")
data = loader.load()

In [ ]:
openai_embeddings = OpenAIEmbeddings()

In [ ]:
vector_store = Chroma.from_documents(data, embedding=openai_embeddings, persist_directory="../../Downloads/vectorstores")
vector_store.persist()